In [1]:
import torch
X = torch.tensor([[1, 0], [0, 1]])
Y = torch.tensor([[2, 1], [1, 2]]) 
X_times_Y = X * Y

In [2]:
X_times_Y

tensor([[2, 0],
        [0, 2]])

### Working with images

#### Loading an Image file

In [2]:
# using imageio
import imageio.v3 as iio

img_io = iio.imread('/home/mahfuz/Desktop/dlpyt/p1ch4/cat1.png')
img_io.shape

(256, 256, 3)

In [3]:
import cv2 as cv
print(cv.__version__)

4.6.0


In [4]:
import cv2 as cv
img_cv = cv.imread('/home/mahfuz/Desktop/dlpyt/p1ch4/cat1.png')
img_cv.shape
cv.imshow('image show', img_cv)
cv.waitKey(0)
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread (0x2cd11b0).
Cannot move to target thread (0x33cd220)

QObject::moveToThread: Current thread (0x33cd220) is not the object's thread

In [5]:
# using imageio
import imageio.v3 as iio

img_io = iio.imread('/home/mahfuz/Desktop/dlpyt/p1ch4/cat1.png')
img_io.shape

(256, 256, 3)

In [6]:
import torch
import numpy as np
img = torch.from_numpy(img_io)
out = img.permute(2, 0, 1)

/home/mahfuz/anaconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_6608/4162671256.py:3: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642969563/work/torch/csrc/utils/tensor_numpy.cpp:199.)
  img = torch.from_numpy(img_io)


In [7]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [8]:
import os

data_dir = '/home/mahfuz/Desktop/dlpyt/p1ch4/'
file_names = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
file_names

['cat1.png', 'cat3.png', 'cat2.png']

In [9]:
for i, file_names in enumerate(file_names):
    img_arr = iio.imread(os.path.join(data_dir, file_names))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

#### Normalizing the data
Neural networks exhibit the best training performance when the input
data ranges roughly from 0 to 1, or from -1 to 1 (this is an effect of how their building
blocks are defined).

In [10]:
batch = batch.float()
batch /= 255.0

Another possibility is to compute the mean and standard deviation of the input data
and scale it so that the output has zero mean and unit standard deviation across each
channel:

In [11]:
n_channels = batch.shape[1]
n_channels

3

In [12]:
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

#### Loading a specialized format

In [13]:
import imageio
sp_dir_path = '/home/mahfuz/Desktop/dlwpt-code-master/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
vol_arr = imageio.volread(sp_dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 46/99  (46.578/99  (78.899/99  (100.0%)


(99, 512, 512)

As was true in section 4.1.3, the layout is different from what PyTorch expects, due to
having no channel information. So we’ll have to make room for the channel dimen-
sion using unsqueeze :

In [14]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 99, 512, 512])

### Representing tabular data

In [15]:
import csv 
import numpy as np 
wine_path = '/home/mahfuz/Desktop/dlwpt-code-master/data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [16]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

proceed to convert the NumPy array to a PyTorch tensor:

In [17]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [20]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [21]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

If we want to transform the target tensor in a tensor of labels, we have two options,
depending on the strategy or what we use the categorical data for. One is simply to
treat labels as an integer vector of scores:

In [22]:
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])